# Multi-GPU Batch Betweenness Centrality
#### Author : Xavier Cadet
In this notebook, we will compute Betweenness Centrality for vertices using cuGraph and will see how to **use Multiple GPUs to compute Betweenness Centrality scores**.

This notebook was tested using 4 NVIDIA Tesla V100-DGX 32G GPUs, using RAPIDS 0.15, and CUDA 10.1. Please be aware that your system may be different and you may need to modify the code or install packages to run the below examples. If you think you have found a bug or an error, please file an issue in [cuGraph](https://github.com/rapidsai/cugraph/issues)

## Introduction
Betweennes Centrality can be slow to compute on large graphs, in order to speed up the process we can leverage multiple GPUs.
In this notebook we will showcase how it would have been done with a Single GPU approach, then we will show how it can be done using multiple GPUs.

## Data
The soc-LiveJournal1 dataset which can be obtained on [SNAP](https://snap.stanford.edu/data/soc-LiveJournal1.html). This graph contains roughly 5 million nodes, and 70 million edges and was extracted from the LiveJournal online social network, further information can be found in:

*Group Formation in Large Social Networks: Membership, Growth, and Evolution., L. Backstrom, D. Huttenlocher, J. Kleinberg, X. Lan., KDD, 2006.*

and:

*Community Structure in Large Networks: Natural Cluster Sizes and the Absence of Large Well-Defined Clusters., J. Leskovec, K. Lang, A. Dasgupta, M. Mahoney., Internet Mathematics 6(1) 29--123, 2009.*

## Betweenness Centrality with cuGraph

### The imports:

In [1]:
import cugraph
import cudf

import dask
import dask_cuda
import cugraph.comms as Comms

In [2]:
import time
import cupy

## Get the data


In [3]:
import urllib.request
import os

data_dir = '../data/'
if not os.path.exists(data_dir):
    print('creating data directory')
    os.system('mkdir ../data')

In [4]:
# download the soc-LiveJournal1 dataset
base_url = 'https://snap.stanford.edu/data/'
fn = 'soc-LiveJournal1.txt'
comp = '.gz'
if not os.path.isfile(data_dir + fn):
    if not os.path.isfile(data_dir + fn + comp):
        print(f'Downloading {base_url + fn + comp} to {data_dir + fn + comp}')
        urllib.request.urlretrieve(base_url + fn + comp, data_dir + fn + comp)
    print(f'Decompressing {data_dir + fn + comp}...')
    os.system('gunzip ' + data_dir + fn + comp)
    print(f'{data_dir + fn + comp} decompressed!')
else:
    print(f'Your data file, {data_dir + fn}, already exists')
input_data_path = data_dir + fn

Your data file, ../data/soc-LiveJournal1.txt, already exists


## Single GPU

### Reading the Data - Single GPU
The following shows how we would read the csv file using a single GPU as it is commonly done when using a single GPU with CuGraph.

In [5]:
t_start_read_sg = time.perf_counter()
e_list = cudf.read_csv(input_data_path, delimiter='\t', names=['src', 'dst'], dtype=['int32', 'int32'])
t_stop_read_sg = time.perf_counter()

In [6]:
print("SG Read time: {}s".format(t_stop_read_sg - t_start_read_sg))

SG Read time: 1.4840830960310996s


### Building the Graph - Single GPU
Once we read the file, we need to build the Graph, we will use a DiGraph, and use the content extracted from the .csv file as an edge list.

In [7]:
t_start_build_sg = time.perf_counter()
G = cugraph.DiGraph()
G.from_cudf_edgelist(e_list, source='src', destination='dst')
t_stop_build_sg = time.perf_counter()

In [8]:
print("SG Build time: {}s".format(t_stop_build_sg - t_start_build_sg))

SG Build time: 0.43075192789547145s


### Calling the Algorithm -  Single GPU
Now that our graph is built, we can get its betweenness centrality score. Here we will use a sub-sample of 1024 sources in order to have a better approximation of the overall betweenness centrality. We set the seed for comparability with the multi GPU version that comes next.

In [9]:
t_start_sg = time.perf_counter()
sg_df = cugraph.betweenness_centrality(G, k=1024, seed=123)
t_stop_sg = time.perf_counter()

In [10]:
print("SG Time elapsed: {}s".format(t_stop_sg - t_start_sg))

SG Time elapsed: 45.48291095509194s


## Now let's use multiple GPUs!

### Using a Dask Cluster
In order to use multiple GPU, we need to ensure that we have Dask Cluster and Client running, further more we need to initialize the CuGraph Communicator.

In [11]:
cluster = dask_cuda.LocalCUDACluster()
client = dask.distributed.Client(cluster)
Comms.initialize()

### Enabling Multi GPU Batch Processing
The good thing is that with a simple `enable_mg_batch` call you can harness the power of Multiple GPUs to operate Batch Processing.
This step might take a few seconds, indeed we need to get the graph available to all GPUS, do not worry, this is only required once or when adding new representations to the graph (adjacency list for example)

In [12]:
t_start_mg = time.perf_counter()
G.enable_batch()
print("MG Batch Enabling Time elapsed: {}s".format(time.perf_counter() - t_start_mg))

/home/xcadet/miniconda3/envs/cugraph_dev/lib/python3.7/site-packages/distributed/client.py:3493: RuntimeWarning: coroutine 'Client._update_scheduler_info' was never awaited
  self.sync(self._update_scheduler_info)
/home/xcadet/miniconda3/envs/cugraph_dev/lib/python3.7/site-packages/distributed/client.py:3493: RuntimeWarning: coroutine 'Client._update_scheduler_info' was never awaited
  self.sync(self._update_scheduler_info)
/home/xcadet/miniconda3/envs/cugraph_dev/lib/python3.7/site-packages/distributed/client.py:3493: RuntimeWarning: coroutine 'Client._update_scheduler_info' was never awaited
  self.sync(self._update_scheduler_info)


MG Batch Enabling Time elapsed: 5.887182566104457s


### Calling the algorithm
We call the algorithm the same way as we used to, but this time it is much faster as we leverage multiple GPUs to compute the Betweenness Centrality scores.

In [13]:
t_start_mg = time.perf_counter()
batch_df = cugraph.betweenness_centrality(G, k=1024, seed=123)
t_stop_mg = time.perf_counter()

In [14]:
print("MG Time elapsed: {}s".format(t_stop_mg - t_start_mg))

MG Time elapsed: 11.294120778795332s


### Verification
Order in the DataFrame might vary, but scores for each vertices match, in order to display them side by side we will first sort the resluts based on the `vertex` key, and renew the DataFramee index.

In [15]:
sorted_sg_df = sg_df.sort_values("vertex").reset_index(drop=True)
sorted_batch_df = batch_df.sort_values("vertex").reset_index(drop=True)

We can now compare score for each of the vertices:

In [16]:
cupy.allclose(sorted_sg_df["betweenness_centrality"], sorted_batch_df["betweenness_centrality"])

True

And just to visually compare the results we can display the DataFrames:

In [17]:
print(sorted_sg_df)

         betweenness_centrality   vertex
0                  4.586460e-06        0
1                  1.715658e-04        1
2                  1.361555e-05        2
3                  7.880936e-06        3
4                  8.486780e-06        4
...                         ...      ...
4847566            0.000000e+00  4847566
4847567            0.000000e+00  4847567
4847568            9.186307e-07  4847568
4847569            0.000000e+00  4847569
4847570            4.984750e-08  4847570

[4847571 rows x 2 columns]


In [18]:
print(sorted_batch_df)

         betweenness_centrality   vertex
0                  4.586460e-06        0
1                  1.715658e-04        1
2                  1.361555e-05        2
3                  7.880936e-06        3
4                  8.486780e-06        4
...                         ...      ...
4847566            0.000000e+00  4847566
4847567            0.000000e+00  4847567
4847568            9.186307e-07  4847568
4847569            0.000000e+00  4847569
4847570            4.984750e-08  4847570

[4847571 rows x 2 columns]


Do not forget to clear the Communicator / client /cluster if required.

In [19]:
Comms.destroy()
client.close()
cluster.close()

___
Copyright (c) 2020, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___